<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#loading" data-toc-modified-id="loading-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>loading</a></span></li><li><span><a href="#modeling" data-toc-modified-id="modeling-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>modeling</a></span></li><li><span><a href="#pred" data-toc-modified-id="pred-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>pred</a></span></li></ul></div>

In [1]:
import sys
import os
my_module = os.path.abspath("../src/")
sys.path.append(my_module)

In [2]:
from load_data import LoadData
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt


# loading

In [3]:
import uuid

In [4]:
uid = uuid.uuid1()

In [12]:
docs = "../docs/"
info = docs + "info/"
finfo = info + "ddl.csv"

data = "../data/"
data_in = data + "in/"
data_out = data + "out/"
fin = data_in + "train_data.csv"
ftest = data_in + "test_data.csv"
fout = data_out + "svc_%s.csv"%uid

models = "../models/"
mout = models + "svc_%s.pkl"%uid

loader = LoadData(fin, finfo)
X, y = loader.load()

In [6]:
X = X.astype(float)
y = y.astype(float)

# modeling

In [7]:
params = [
#     {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
    {'C': [1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.001, 0.0001]},
#     {'C': [1, 10, 100, 1000], 'kernel': ['poly'], 'degree': [2, 3, 4], 'gamma': [0.001, 0.0001]},
    {'C': [1, 10, 100, 1000], 'kernel': ['sigmoid'], 'gamma': [0.001, 0.0001]}
    ]

In [8]:
cv = GridSearchCV(SVC(), param_grid=params, cv=3, scoring='accuracy')

In [9]:
%%time
cv.fit(X, y)

Wall time: 1h 25min 55s


GridSearchCV(cv=3, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.001, 0.0001]}, {'C': [1, 10, 100, 1000], 'kernel': ['sigmoid'], 'gamma': [0.001, 0.0001]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [10]:
cv.best_params_

{'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}

In [11]:
cv.best_score_

0.7796666666666666

In [14]:
from sklearn.externals import joblib
clf = SVC(cv.best_params_)


joblib.dump(clf, mout)

['../models/svc_013b7c9e-5d4e-11e9-ab25-f06e0bb794dc.pkl']

# pred

In [17]:
loader = LoadData(ftest, finfo)
test_X= loader.load()

In [18]:
test_X = test_X.astype(float)

In [19]:
pred = cv.predict(test_X)

In [27]:
test_X["Y"] = pred

In [28]:
test_X["Y"].to_csv(fout, header=True)

In [26]:
y.astype(float).sum() / y.count()

0.222